In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [62]:
i = -0.4
j = -0.1
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX_CODE',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})", name='Texas100', sensitive_column='ETHNICITY'
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}}


100%|██████████| 2/2 [00:00<00:00,  3.68it/s]


{0: {(0, 1): 6250, (0, 0): 6250, (1, 1): 6250, (1, 0): 6250}, 1: {(0, 1): 5625, (0, 0): 6875, (1, 1): 6875, (1, 0): 5625}}


100%|██████████| 2/2 [00:00<00:00,  4.31it/s]


In [68]:
hidden_layer_size_tuples = [(64, 32, 16, 8), (32, 16, 8), (16, 8)]
experiment.clfs={}

for i, hidden_layer_size_tuple in enumerate(hidden_layer_size_tuples[1:2]):
    base_model = model_utils.get_model(max_iter=500, hidden_layer_sizes=hidden_layer_size_tuple)
    clf = copy.deepcopy(base_model)
    clf.fit(experiment.X_test, experiment.y_te_onehot)
    experiment.clfs[hidden_layer_size_tuple] = clf
    

In [49]:
save_model = True

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42, max_depth=7)
clf.fit(experiment.X_test, experiment.y_te_onehot)
experiment.clfs['DT'] = clf

In [10]:
experiment.ds.ds.original_df.columns

Index(['SEX_CODE', 'TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION',
       'LENGTH_OF_STAY', 'PAT_AGE', 'PAT_STATUS', 'RACE', 'ETHNICITY',
       'TOTAL_CHARGES', 'ADMITTING_DIAGNOSIS', 'PRINC_SURG_PROC_CODE',
       'is_train'],
      dtype='object')

In [51]:
n

24999.0

In [65]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==0].copy().reset_index(drop=True).drop(['is_train'], axis=1)

p = -0.1
x = original_test_df[original_test_df['SEX_CODE']==0][['ETHNICITY', 'PRINC_SURG_PROC_CODE']].value_counts().to_numpy().min()
n = (x * 4) // (1 + p)

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p1=-0.1, p2=-0.1, n=2*n, subgroup_col_name='SEX_CODE', transformed_already=True, return_indices_only=True)

experiment.X_test_balanced_corr = experiment.X_test.loc[temp_indices].reset_index(drop=True)
experiment.y_te_balanced_corr = experiment.y_te[temp_indices]
experiment.y_te_onehot_balanced_corr = experiment.y_te_onehot[temp_indices]

[0, 1]
[16666.0, 16666.0]
{0: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}, 1: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}}


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 32.69it/s]


In [52]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==0].copy().reset_index(drop=True).drop(['is_train'], axis=1)

p = -0.1
x = original_test_df[original_test_df['SEX_CODE']==1][['ETHNICITY', 'PRINC_SURG_PROC_CODE']].value_counts().to_numpy().min()
n = (x * 4) // (1 + p)

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p=[-0.1, -0.1], n=[n, n], subgroup_col_name='SEX_CODE', transformed_already=True, return_indices_only=True)

experiment.X_test_balanced_corr = experiment.X_test.loc[temp_indices].reset_index(drop=True)
experiment.y_te_balanced_corr = experiment.y_te[temp_indices]
experiment.y_te_onehot_balanced_corr = experiment.y_te_onehot[temp_indices]

[0, 1]
[16666.0, 16666.0]
{0: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}, 1: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}}


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 31.34it/s]


In [66]:
hidden_layer_size_tuples = [(64, 32, 16, 8), (32, 16, 8), (16, 8)]
experiment.clfs_balanced_corr={}

for i, hidden_layer_size_tuple in enumerate(hidden_layer_size_tuples[1:2]):
    base_model = model_utils.get_model(max_iter=500, hidden_layer_sizes=hidden_layer_size_tuple)
    experiment.clfs_balanced_corr[hidden_layer_size_tuple] = copy.deepcopy(base_model)
    experiment.clfs_balanced_corr[hidden_layer_size_tuple].fit(experiment.X_test_balanced_corr, experiment.y_te_onehot_balanced_corr)

In [54]:
clf = DecisionTreeClassifier(random_state=42, max_depth=7)
clf.fit(experiment.X_test_balanced_corr, experiment.y_te_onehot_balanced_corr)
experiment.clfs_balanced_corr['DT'] = clf

In [79]:
def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'SEX_CODE', indices=None):
    if isinstance(clf, DecisionTreeClassifier):
        y_tr_pred = np.argmax(clf.predict_proba(X_train)[1], axis=1)
    else:
        y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    subgroup_vals_tr = X_train[f'{subgroup_col_name}_1'].to_numpy().ravel().astype(int)
    subgroup_vals_te = X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    sens_pred_LOMIA = LOMIA_attack(experiment, clf, X_test, y_te, experiment.ds.ds.meta, indices=indices)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())
    correct_indices_LOMIA = (sens_pred_LOMIA == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

    male_indices = np.where(subgroup_vals_tr==0)[0]
    female_indices = np.where(subgroup_vals_tr==1)[0]
    print(correct_indices_LOMIA[np.where(subgroup_vals_te==0)[0]].mean())
    print(correct_indices_LOMIA[np.where(subgroup_vals_te==1)[0]].mean())
    perf_dict = {
        'ASRD_CSMIA': round(100 * np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'ASRD_LOMIA': round(100 * np.ptp([correct_indices_LOMIA[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'EOD': round(equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'DPD': round(demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'MA_Male': 100 * accuracy_score(y_tr.ravel()[male_indices], y_tr_pred[male_indices]),
        'MA_Female': 100 * accuracy_score(y_tr.ravel()[female_indices], y_tr_pred[female_indices])
    }

    return perf_dict


In [80]:
model_arch = hidden_layer_size_tuples[1]
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr),
    # 'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr)
    'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices)
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

0.7008
0.55632
0.5586223448937957
0.5834633385335414


In [69]:
model_arch = hidden_layer_size_tuples[1]
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr),
    'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices)
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

0.50224
0.5340213608544342


In [81]:
res_dict_df

,ASRD_CSMIA,ASRD_LOMIA,EOD,DPD,MA_Male,MA_Female
w/o BCorr,12.12,14.45,0.0332,0.0177,72.528,74.080
w Bcorr,0.94,2.48,0.0221,0.0186,74.696,74.348


In [82]:
print(res_dict_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & ASRD_CSMIA & ASRD_LOMIA & EOD & DPD & MA_Male & MA_Female \\
\midrule
w/o BCorr & 12.120000 & 14.450000 & 0.033200 & 0.017700 & 72.528000 & 74.080000 \\
w Bcorr & 0.940000 & 2.480000 & 0.022100 & 0.018600 & 74.696000 & 74.348000 \\
\bottomrule
\end{tabular}



In [35]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr, subgroup_col_name='SEX_CODE'))

{'EOD': 0.023279999999999967, 'DPD': 0.008360000000000034, 'ASRD': 0.10883999999999994}
{'EOD': 0.03320000000000001, 'DPD': 0.017679999999999974, 'ASRD': 0.12124000000000001}
{'EOD': 0.03159999999999999, 'DPD': 0.0016800000000000148, 'ASRD': 0.1444399999999999}
{'EOD': 0.14151999999999998, 'DPD': 0.09380000000000005, 'ASRD': 0.13656000000000001}


In [36]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, subgroup_col_name='SEX_CODE'))

{'EOD': 0.012799999999999978, 'DPD': 0.012440000000000007, 'ASRD': 0.012241958713394063}
{'EOD': 0.028240000000000015, 'DPD': 0.024920000000000053, 'ASRD': 0.0032405184829572553}
{'EOD': 0.005520000000000025, 'DPD': 0.00548000000000004, 'ASRD': 0.009961593855016826}
{'EOD': 0.07272, 'DPD': 0.03423999999999999, 'ASRD': 0.01524243879020648}


In [12]:
get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, {'SEX': 0})

0.10008400408079707